# PPL 

# Model interpretability via SHAP values

<br>
<hr>

<br>

<h1 style="color:red"> Index</h1>




## 0. Setup
## 1. Dataset Selection
## 2. Model Selection
## 3. Calculate SHAP values
## 4. Visualization
- ### 4.1 Explainability at a Local Level (specific project)
- ### 4.2 Explainability at a Global Level
- ### 4.3 Importance Ranking
- ### 4.4 Explainability between Features

<br>
<hr>
<br>
<br>

## 0. Setup

In [ ]:
# set root folder (needed for imports)
import os
while not os.getcwd().endswith("ppl-model"):
    os.chdir("..")


# larger page width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# detect changes on folders
%load_ext autoreload
%autoreload 2

# ignore warning on jupyter
import warnings
warnings.filterwarnings('ignore')

# my imports
import settings
from src.modeling import util
from src.modeling.models import nnet
from dskc import dskc_terminal
from dskc import dskc_util

# third party imports
import pandas as pd
import numpy as np
import shap
import pickle

# see all rows of pandas dataframe
pd.set_option('display.max_rows', None) # option to see all rows
pd.set_option('display.max_columns', None)  # option to see all columns
pd.options.display.float_format = '{:,}'.format #to display floats with comma separators 


<br>

## 1. Dataset Selection

In [ ]:
x_train, y_train, feature_names = util.read_train_data()
x_test, y_test, test_feature_names = util.read_test_data()
print(x_train.shape)
print(x_test.shape)
x_test_df = pd.DataFrame(x_test[:1000],columns=feature_names)

<br> 

## 2. Model Selection

In [ ]:
model = nnet.load_model()


<br>

## 3. Calculate SHAP values

### Set explainer

In [ ]:
bg =  x_train[:20000]

# set explainer
print("\nCalculating shap explainer...") 
timer = dskc_util.Timer()
shap_explainer = shap.DeepExplainer(model,bg)
timer.end()

# save
with open('bg.pkl', 'wb') as f:
    pickle.dump(bg, f)
    
# explainer expected value
expected_value = shap_explainer.expected_value[0]


<br>

### Get SHAP values

In [ ]:
# calc shap
print("\nCalculating shap values for test...") 
timer = dskc_util.Timer()
shap_values = shap_explainer.shap_values(x_test[:1000])
timer.end()


<br>

## 4. Visualization


The probabilty predicted by the model can be expressed with the following formula:

$$P(\textit{financed}) = \textit{base value} + \sum_{i=1}^{74} SHAP_{i}$$


A very intuive explanation can be found [here](https://medium.com/@gabrieltseng/interpreting-complex-models-with-shap-values-1c187db6ec83).


In [ ]:
print("SHAP Base value", expected_value)

<br>

### 4.1 Explainability at a Local Level (specific project)

The explanation below shows for a **single entry** the contributing features to push the model output from the **base value** (the average model output over the training dataset we passed) towards one. **For this entry there are features pushing** the class label **up**, they are shown in red. And there also are features pushing the class label **down**, they are shown in blue.

In [ ]:
# set project index
index = 10

# init the JS visualization code
shap.initjs()

# predticion value
predicted_value = model.predict(x_test)[index][0]
print("Predicted value:",predicted_value)

# plot the explanation of the first prediction
# Note the model is "multi-output" because it is rank-2 but only has one column
shap.force_plot(shap_explainer.expected_value[0], shap_values[0][index], feature_names)


<br>

#### Detailed SHAP values (ascending order)

In [ ]:
aux = pd.DataFrame(shap_values[0][index], index=feature_names, columns=['SHAP Values']).sort_values(by=['SHAP Values'], ascending=False)
aux

<br>

### 4.2 Explainability at a Global Level

In [ ]:
# visualize the training set predictions
shap.force_plot(shap_explainer.expected_value, shap_values[0], x_test_df)

<br>

### 4.3 Importance Ranking

We can also just take the mean absolute value of the SHAP values for each feature to get a standard bar plot.

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, x_test_df)

<br>

### 4.4 Explainability between Features


To understand how a single feature affects the output of the model we can plot the SHAP value of that feature vs the value of the feature for all the examples in the dataset. Since SHAP values represent a feature's responsibility for a change in the model output, the plot below represents the change in `be financed probability` as feature changes. 

Vertical dispersion at a single level represents interaction effects with other features. To help reveal these interactions, we used a second feature for for coloring. 

<br>

In [ ]:
# select features
feature_1 = "percentage_days_elapsed"
feature_2 = "percentage_raised"

# plot
shap.dependence_plot(feature_1, shap_values[0], x_test_df[:1000],interaction_index =feature_2)

In [ ]:
# select features
feature_1 = "percentage_raised"
feature_2 = "channel"

# plot
shap.dependence_plot(feature_1, shap_values[0], x_test_df[:1000],interaction_index =feature_2)